In [18]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import shutil

# Paths
images_dir = '/kaggle/input/cars-video-object-tracking/images'
boxes_dir = '/kaggle/input/cars-video-object-tracking/boxes'
output_dir = '/kaggle/working/dataset'

# YOLO output structure
for folder in ['images/train', 'images/val', 'images/test', 'labels/train', 'labels/val', 'labels/test']:
    os.makedirs(os.path.join(output_dir, folder), exist_ok=True)

# List image and box files
image_files = [f for f in os.listdir(images_dir) if f.endswith('.PNG')]
box_files = [f for f in os.listdir(boxes_dir) if f.endswith('.PNG')]

# Align filenames between images and boxes
matched_files = set(image_files).intersection(box_files)
image_files = [f for f in image_files if f in matched_files]
box_files = [f for f in box_files if f in matched_files]

# Ensure files are not empty
if len(image_files) == 0 or len(box_files) == 0:
    raise ValueError("No matching image or box files found. Check your directories.")

# Split dataset
train_images, temp_images, train_boxes, temp_boxes = train_test_split(
    image_files, box_files, test_size=0.3, random_state=42
)
val_images, test_images, val_boxes, test_boxes = train_test_split(
    temp_images, temp_boxes, test_size=0.5, random_state=42
)

# Debug splits
print(f"Train size: {len(train_images)}, Validation size: {len(val_images)}, Test size: {len(test_images)}")

# Function to convert box PNG to YOLO format
def convert_box_to_yolo(image_path, box_path, output_label_path):
    # Load the image and box files
    img = cv2.imread(image_path)
    box_img = cv2.imread(box_path, cv2.IMREAD_GRAYSCALE)  # Grayscale for processing

    # Get image dimensions
    height, width = img.shape[:2]

    # Find contours in the box image
    contours, _ = cv2.findContours(box_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Open the label file for writing
    with open(output_label_path, 'w') as f:
        for contour in contours:
            # Get bounding box for each contour
            x, y, w, h = cv2.boundingRect(contour)

            # Calculate YOLO format values
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            w_norm = w / width
            h_norm = h / height

            # Write to file (class_id is set to 0 for now; modify for multiple classes)
            f.write(f"0 {x_center} {y_center} {w_norm} {h_norm}\n")

# Function to move files and generate labels
def process_files(image_files, box_files, img_dst, lbl_dst):
    for img_file, box_file in zip(image_files, box_files):
        # Paths
        img_path = os.path.join(images_dir, img_file)
        box_path = os.path.join(boxes_dir, box_file)
        label_path = os.path.join(lbl_dst, img_file.replace('.PNG', '.txt'))

        # Copy image to destination
        shutil.copy(img_path, os.path.join(img_dst, img_file))

        # Convert boxes to YOLO format
        convert_box_to_yolo(img_path, box_path, label_path)

# Process train, val, and test sets
process_files(train_images, train_boxes, f'{output_dir}/images/train', f'{output_dir}/labels/train')
process_files(val_images, val_boxes, f'{output_dir}/images/val', f'{output_dir}/labels/val')
process_files(test_images, test_boxes, f'{output_dir}/images/test', f'{output_dir}/labels/test')

# Create YAML file
yaml_content = f"""
train: {output_dir}/images/train
val: {output_dir}/images/val
test: {output_dir}/images/test

nc: 1  # Update with the number of classes if more than one
names: ['car']  # Update with class names
"""


Train size: 210, Validation size: 45, Test size: 46


In [20]:
with open('/kaggle/working/dataset.yaml', 'w') as f:
    f.write(yaml_content)

# Train YOLOv8 model
model = YOLO('yolov8n.pt')  # Load pre-trained YOLOv8 model
model.train(data='/kaggle/working/dataset.yaml', epochs=100, imgsz=640)

Ultralytics 8.3.50 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /kaggle/working/dataset/labels/train... 210 images, 0 backgrounds, 0 corrupt: 100%|██████████| 210/210 [00:04<00:00, 45.50it/s]

train: New cache created: /kaggle/working/dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/dataset/labels/val... 45 images, 0 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<00:00, 50.67it/s]

val: New cache created: /kaggle/working/dataset/labels/val.cache


...

                   all         45         45      0.999          1      0.995      0.995



100 epochs completed in 0.160 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.50 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.65it/s]


                   all         45         45      0.999          1      0.995      0.995
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c1f5e6353c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [31]:

import zipfile
import os
import shutil

# Paths
labels_dir = '/kaggle/working/dataset/labels/test'
images_dir = '/kaggle/working/dataset/images/test'
zip_labels_path = '/kaggle/working/labels_test.zip'
zip_images_path = '/kaggle/working/images_test.zip'
local_labels_path = 'C:/Users/ignat/Desktop/labels_test.zip'
local_images_path = 'C:/Users/ignat/Desktop/images_test.zip'

# Create ZIP files
with zipfile.ZipFile(zip_labels_path, 'w') as zipf:
    for root, dirs, files in os.walk(labels_dir):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), labels_dir))

with zipfile.ZipFile(zip_images_path, 'w') as zipf:
    for root, dirs, files in os.walk(images_dir):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), images_dir))


if not os.path.exists(local_labels_path):
    shutil.move(zip_labels_path, local_labels_path)
else:
    print(f"{local_labels_path} already exists. Overwriting...")

# Move images zip
if not os.path.exists(local_images_path):
    shutil.move(zip_images_path, local_images_path)
else:
    print(f"{local_images_path} already exists. Overwriting...")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/ignat/Desktop/labels_test.zip'